# MinIO + DuckDB SPY 1-min Dashboard
This notebook reads daily SPY 1-minute bars directly from MinIO (S3-compatible) using DuckDB httpfs and plots a candlestick + volume chart.

In [1]:
# Imports and Plotly renderer
import os
import duckdb
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = os.getenv('PLOTLY_RENDERER', 'plotly_mimetype')  # 'vscode' also works in VS Code

In [2]:
# Configure DuckDB httpfs from environment (.env)
S3_ENDPOINT = (os.getenv('S3_ENDPOINT_URL', '127.0.0.1:9100').replace('http://','').replace('https://',''))
S3_REGION = os.getenv('S3_REGION', 'us-east-1')
S3_USE_SSL = os.getenv('S3_USE_SSL', 'false')
S3_AK = os.getenv('S3_ACCESS_KEY_ID') or os.getenv('MINIO_ROOT_USER', 'minioadmin')
S3_SK = os.getenv('S3_SECRET_ACCESS_KEY') or os.getenv('MINIO_ROOT_PASSWORD', 'minioadmin')
BUCKET = os.getenv('MINIO_BUCKET', 'antman-lake')

con = duckdb.connect()
con.execute('INSTALL httpfs; LOAD httpfs;')
con.execute("SET s3_url_style='path';")
con.execute(f"SET s3_use_ssl='{S3_USE_SSL}';")
con.execute(f"SET s3_region='{S3_REGION}';")
con.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
con.execute("SET s3_access_key_id=$1;", [S3_AK])
con.execute("SET s3_secret_access_key=$1;", [S3_SK])
print('Configured DuckDB httpfs for MinIO at', S3_ENDPOINT)
print('Bucket:', BUCKET)

Configured DuckDB httpfs for MinIO at localhost:9100
Bucket: antman-lake


In [3]:
# Helpers: data loader + TA utilities
import numpy as np

# Load one trading day from MinIO via DuckDB httpfs
def load_day(dt_str: str, ticker: str = 'SPY') -> pd.DataFrame:
    path = f"s3://{BUCKET}/silver/symbol={ticker}/resolution=1min/dt={dt_str}/*.parquet"
    try:
        df = con.execute("SELECT * FROM read_parquet($1, filename=true)", [path]).df()
    except Exception as e:
        print('read_parquet failed:', e)
        return pd.DataFrame()
    # Normalize timestamp column
    ts_col = None
    for c in ['ts','utc_timestamp','timestamp','datetime']:
        if c in df.columns:
            ts_col = c; break
    if ts_col is None:
        return pd.DataFrame()
    df['ts'] = pd.to_datetime(df[ts_col], utc=True, errors='coerce')
    kept = [c for c in ['ts','open','high','low','close','volume'] if c in df.columns]
    df = df[kept].dropna(subset=['ts']).sort_values('ts').reset_index(drop=True)
    df['ts'] = df['ts'].dt.round('ms')
    return df

# TA helpers

def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False).mean()

def bollinger(s: pd.Series, n: int = 20, k: float = 2.0):
    m = s.rolling(n).mean()
    sd = s.rolling(n).std(ddof=0)
    return m, m + k*sd, m - k*sd

def rsi(s: pd.Series, n: int = 14) -> pd.Series:
    d = s.diff()
    up = d.clip(lower=0)
    dn = -d.clip(upper=0)
    roll_up = up.ewm(alpha=1/n, adjust=False).mean()
    roll_dn = dn.ewm(alpha=1/n, adjust=False).mean()
    rs = np.where(roll_dn==0, np.nan, roll_up/roll_dn)
    return pd.Series(100 - (100/(1+rs)), index=s.index)

# ATR helper (Wilder's ATR)

def atr(df: pd.DataFrame, n: int = 14) -> pd.Series:
    if not {'high','low','close'}.issubset(df.columns) or df.empty:
        return pd.Series(dtype='float64')
    prev_close = df['close'].shift(1)
    tr1 = (df['high'] - df['low']).abs()
    tr2 = (df['high'] - prev_close).abs()
    tr3 = (df['low'] - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    # Wilder's smoothing using EMA alpha=1/n
    return tr.ewm(alpha=1/n, adjust=False).mean()

# Fibonacci helpers
FIB_RATIOS = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
FIB_NAMES = ['0%', '23.6%', '38.2%', '50%', '61.8%', '78.6%', '100%']

def compute_fib_levels(df: pd.DataFrame, direction: str = 'auto'):
    if 'high' not in df.columns or 'low' not in df.columns or df.empty:
        return None
    idx_low = df['low'].idxmin()
    idx_high = df['high'].idxmax()
    lo = float(df.loc[idx_low, 'low'])
    hi = float(df.loc[idx_high, 'high'])
    ts_lo = df.loc[idx_low, 'ts'] if 'ts' in df.columns else None
    ts_hi = df.loc[idx_high, 'ts'] if 'ts' in df.columns else None

    if direction == 'auto':
        direction = 'up' if idx_low < idx_high else 'down'

    levels = []
    if direction == 'up':
        for r, name in zip(FIB_RATIOS, FIB_NAMES):
            lvl = lo + (hi - lo) * r
            levels.append((name, lvl))
    else:
        for r, name in zip(FIB_RATIOS, FIB_NAMES):
            lvl = hi - (hi - lo) * r
            levels.append((name, lvl))
    return {
        'direction': direction,
        'low': lo,
        'high': hi,
        'ts_low': ts_lo,
        'ts_high': ts_hi,
        'levels': levels,
    }

In [4]:
# Load a sample date
dt = os.getenv('SAMPLE_DT', '2025-03-11')
df = load_day(dt)
print(f'Loaded {len(df)} rows for {dt}')
df.head()

Loaded 390 rows for 2025-03-11


,ts,open,high,low,close,volume
0,2025-03-11 14:30:00+00:00,555.950,557.24,555.9000,556.630,318164
1,2025-03-11 14:31:00+00:00,556.625,557.03,556.4928,556.600,142751
2,2025-03-11 14:32:00+00:00,556.570,556.71,556.2800,556.310,111371
3,2025-03-11 14:33:00+00:00,556.300,556.50,556.1087,556.435,126317
4,2025-03-11 14:34:00+00:00,556.450,556.48,555.8000,556.110,157529


In [5]:
# Plot candlestick + volume
if not df.empty:
    fig = go.Figure(data=[go.Candlestick(x=df['ts'], open=df.get('open'), high=df.get('high'), low=df.get('low'), close=df.get('close'), name='SPY')])
    if 'volume' in df.columns:
        fig.add_trace(go.Bar(x=df['ts'], y=df['volume'], name='Volume', yaxis='y2', marker_color='lightgray', opacity=0.4))
        fig.update_layout(yaxis2=dict(overlaying='y', side='right', showgrid=False, range=[0, float(df['volume'].max())*4]))
    fig.update_layout(title=f'SPY 1-min {dt}', xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False, template='plotly_white', height=600)
    fig.show()
else:
    print('No data to plot; try another dt or check MinIO path.')

## Tips
- MinIO console: http://localhost:9101
- Ensure `.env` has: `MINIO_BUCKET`, `S3_ENDPOINT_URL=127.0.0.1:9100`, and credentials.
- Weekends/holidays may return empty days; pick a trading day.

## Interactive TA dashboard (EMAs, Bollinger, RSI, MACD)

Pick a trading day and render a multi-panel chart using data loaded directly from MinIO via DuckDB.

In [6]:
import numpy as np
import ipywidgets as W
from IPython.display import display
import traceback
import plotly.io as pio

# TA helpers

def ema(s, span):
    return s.ewm(span=span, adjust=False).mean()

def bollinger(s, n=20, k=2.0):
    m = s.rolling(n).mean()
    sd = s.rolling(n).std(ddof=0)
    return m, m + k*sd, m - k*sd

def rsi(s, n=14):
    d = s.diff()
    up = d.clip(lower=0)
    dn = -d.clip(upper=0)
    roll_up = up.ewm(alpha=1/n, adjust=False).mean()
    roll_dn = dn.ewm(alpha=1/n, adjust=False).mean()
    rs = np.where(roll_dn==0, np.nan, roll_up/roll_dn)
    rsi = 100 - (100/(1+rs))
    return pd.Series(rsi, index=s.index)

# Fibonacci helpers
FIB_RATIOS = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
FIB_NAMES = ['0%', '23.6%', '38.2%', '50%', '61.8%', '78.6%', '100%']


def compute_fib_levels(df, direction='auto'):
    if 'high' not in df.columns or 'low' not in df.columns:
        return None
    # Determine swing using day low/high; decide direction based on which came first
    idx_low = df['low'].idxmin()
    idx_high = df['high'].idxmax()
    lo = float(df.loc[idx_low, 'low'])
    hi = float(df.loc[idx_high, 'high'])
    ts_lo = df.loc[idx_low, 'ts'] if 'ts' in df.columns else None
    ts_hi = df.loc[idx_high, 'ts'] if 'ts' in df.columns else None

    if direction == 'auto':
        direction = 'up' if idx_low < idx_high else 'down'

    levels = []
    if direction == 'up':
        # Move: low -> high; retracements from high downward
        for r, name in zip(FIB_RATIOS, FIB_NAMES):
            lvl = lo + (hi - lo) * r
            levels.append((name, lvl))
    else:
        # Move: high -> low; retracements from low upward
        for r, name in zip(FIB_RATIOS, FIB_NAMES):
            lvl = hi - (hi - lo) * r
            levels.append((name, lvl))
    return {
        'direction': direction,
        'low': lo,
        'high': hi,
        'ts_low': ts_lo,
        'ts_high': ts_hi,
        'levels': levels,
    }

print(f"[setup] plotly renderer={pio.renderers.default}")

# UI widgets
_dt = W.Text(value=os.getenv('SAMPLE_DT', '2025-03-11'), description='Date (YYYY-MM-DD):')
_ema_fast = W.IntSlider(value=9, min=3, max=50, step=1, description='EMA fast')
_ema_slow = W.IntSlider(value=21, min=5, max=100, step=1, description='EMA slow')
_bb_n = W.IntSlider(value=20, min=10, max=50, step=1, description='BB n')
_bb_k = W.FloatSlider(value=2.0, min=1.0, max=3.0, step=0.1, description='BB k')
_fib_on = W.Checkbox(value=True, description='Show Fibonacci')
_fib_dir = W.Dropdown(options=['auto', 'up', 'down'], value='auto', description='Fib dir')

_out = W.Output()


def render(*_):
    with _out:
        _out.clear_output(wait=True)
        try:
            print(f"[render] dt={_dt.value}")
            df = load_day(_dt.value)
            print(f"[render] loaded rows={len(df)} cols={list(df.columns)}")
            if df.empty:
                print('No data for', _dt.value)
                return
            # Indicators
            if 'close' in df.columns:
                df['ema_fast'] = ema(df['close'], _ema_fast.value)
                df['ema_slow'] = ema(df['close'], _ema_slow.value)
                m, up, dn = bollinger(df['close'], _bb_n.value, _bb_k.value)
                df['bb_mid'] = m
                df['bb_up'] = up
                df['bb_dn'] = dn
                r = rsi(df['close'], 14)
                print(
                    f"[render] indicators ok: ema_fast={_ema_fast.value}, ema_slow={_ema_slow.value}, bb_n={_bb_n.value}, bb_k={_bb_k.value}"
                )
                print(f"[render] ts range: {df['ts'].min()} -> {df['ts'].max()}")
            else:
                print('Missing close column; columns=', list(df.columns))
                return
            # Price pane
            fig = go.Figure()
            fig.add_trace(
                go.Candlestick(
                    x=df['ts'],
                    open=df.get('open'),
                    high=df.get('high'),
                    low=df.get('low'),
                    close=df.get('close'),
                    name='Price',
                )
            )
            fig.add_trace(
                go.Scatter(x=df['ts'], y=df['ema_fast'], name=f'EMA{_ema_fast.value}', line=dict(width=1))
            )
            fig.add_trace(
                go.Scatter(x=df['ts'], y=df['ema_slow'], name=f'EMA{_ema_slow.value}', line=dict(width=1))
            )
            fig.add_trace(
                go.Scatter(x=df['ts'], y=df['bb_up'], name='BB up', line=dict(width=1, color='gray'))
            )
            fig.add_trace(
                go.Scatter(x=df['ts'], y=df['bb_mid'], name='BB mid', line=dict(width=1, color='lightgray'))
            )
            fig.add_trace(
                go.Scatter(x=df['ts'], y=df['bb_dn'], name='BB dn', line=dict(width=1, color='gray'))
            )

            # Fibonacci overlay
            if _fib_on.value:
                fib = compute_fib_levels(df, direction=_fib_dir.value)
                if fib is None:
                    print('[fib] cannot compute (need high/low columns)')
                else:
                    print(
                        f"[fib] dir={fib['direction']} low={fib['low']:.2f} at {fib['ts_low']} high={fib['high']:.2f} at {fib['ts_high']}"
                    )
                    colors = ['#2ca02c', '#1f77b4', '#9467bd', '#ff7f0e', '#d62728', '#8c564b', '#7f7f7f']
                    for (name, lvl), color in zip(fib['levels'], colors):
                        fig.add_hline(
                            y=lvl,
                            line_dash='dot',
                            line_color=color,
                            annotation_text=f"Fib {name} {lvl:.2f}",
                            annotation_position='right',
                            opacity=0.6,
                        )

            # Volume on secondary axis
            if 'volume' in df.columns:
                fig.add_trace(
                    go.Bar(
                        x=df['ts'],
                        y=df['volume'],
                        name='Volume',
                        yaxis='y2',
                        marker_color='lightgray',
                        opacity=0.4,
                    )
                )
                fig.update_layout(
                    yaxis2=dict(
                        overlaying='y', side='right', showgrid=False, range=[0, float(df['volume'].max()) * 4]
                    )
                )
            fig.update_layout(
                title=f'SPY {_dt.value}',
                xaxis_title='Time',
                yaxis_title='Price',
                xaxis_rangeslider_visible=False,
                template='plotly_white',
                height=700,
            )
            display(fig)

            # RSI pane
            fig2 = go.Figure()
            fig2.add_trace(go.Scatter(x=df['ts'], y=r, name='RSI'))
            fig2.add_hline(y=70, line_dash='dot', line_color='red')
            fig2.add_hline(y=30, line_dash='dot', line_color='green')
            fig2.update_layout(height=200, template='plotly_white', title='RSI (14)')
            display(fig2)
        except Exception as e:
            print('[render] error:', repr(e))
            traceback.print_exc()

for w in (_dt, _ema_fast, _ema_slow, _bb_n, _bb_k, _fib_on, _fib_dir):
    w.observe(render, names='value')

# Explicitly display the widget container
_box = W.VBox([
    _dt,
    W.HBox([_ema_fast, _ema_slow, _bb_n, _bb_k]),
    W.HBox([_fib_on, _fib_dir]),
    _out,
])
display(_box)

# Initial render
render()


[setup] plotly renderer=plotly_mimetype


## Signals: Fibonacci Pullback Strategy

This section derives simple buy/sell signals from the Fibonacci retracement overlays and plots them on a dedicated chart. Rules (per side):

- Direction: use intraday swing (auto) to decide up/down.
- Entry: price touches the 50–61.8% zone; trend filter (EMA9 vs EMA21); momentum check (RSI).
- Stop: beyond 78.6% or swing extreme with an ATR buffer.
- Targets: 1R partial; then swing extreme or 127.2%/161.8% extension.

A compact table prints key levels per signal.

In [7]:
# Compute and visualize simple Fibonacci pullback signals for the selected day
import math
from IPython.display import display

try:
    # Reuse the same date and loader from the dashboard
    day = _dt.value if '_dt' in globals() else os.getenv('SAMPLE_DT', '2025-03-11')
    sdf = load_day(day)
    print(f"[signals] dt={day} rows={len(sdf)}")
    if sdf.empty or not {'open','high','low','close'}.issubset(sdf.columns):
        print('[signals] missing data columns or empty day')
    else:
        # Indicators required for filters
        sdf['ema_fast'] = ema(sdf['close'], _ema_fast.value if '_ema_fast' in globals() else 9)
        sdf['ema_slow'] = ema(sdf['close'], _ema_slow.value if '_ema_slow' in globals() else 21)
        rsis = rsi(sdf['close'], 14)
        sdf['rsi'] = rsis

        # Determine swing and fib levels
        fib = compute_fib_levels(sdf, direction=_fib_dir.value if '_fib_dir' in globals() else 'auto')
        if not fib:
            print('[signals] cannot compute fib levels')
        else:
            lo, hi, direction = fib['low'], fib['high'], fib['direction']
            # Key levels
            fib_levels = dict(fib['levels'])
            L50, L618, L786 = fib_levels['50%'], fib_levels['61.8%'], fib_levels['78.6%']

            # Extension levels
            rng = hi - lo
            ext127, ext161 = (hi + 0.272*rng, hi + 0.618*rng) if direction=='up' else (lo - 0.272*rng, lo - 0.618*rng)

            # Build boolean masks
            in_zone = (sdf['low'] <= max(L50,L618)) & (sdf['high'] >= min(L50,L618))
            trend_up = (sdf['ema_fast'] > sdf['ema_slow'])
            trend_dn = (sdf['ema_fast'] < sdf['ema_slow'])
            mom_ok_long = sdf['rsi'] > 45
            mom_ok_short = sdf['rsi'] < 55

            # Entry signals
            long_ok = (direction=='up') & in_zone & trend_up & mom_ok_long
            short_ok = (direction=='down') & in_zone & trend_dn & mom_ok_short

            sdf['long_entry'] = long_ok
            sdf['short_entry'] = short_ok

            # Simple plot
            sfig = go.Figure()
            sfig.add_trace(go.Candlestick(x=sdf['ts'], open=sdf['open'], high=sdf['high'], low=sdf['low'], close=sdf['close'], name='Price'))
            # Zone bands
            sfig.add_hline(y=L50, line_dash='dot', line_color='#999', annotation_text='Fib 50%')
            sfig.add_hline(y=L618, line_dash='dot', line_color='#666', annotation_text='Fib 61.8%')
            sfig.add_hline(y=L786, line_dash='dot', line_color='#444', annotation_text='Fib 78.6%')
            # Extensions
            sfig.add_hline(y=ext127, line_dash='dash', line_color='#2ca02c', annotation_text='127.2%')
            sfig.add_hline(y=ext161, line_dash='dash', line_color='#1f77b4', annotation_text='161.8%')

            # Markers
            if sdf['long_entry'].any():
                e = sdf[sdf['long_entry']]
                sfig.add_trace(go.Scatter(x=e['ts'], y=e['close'], mode='markers', marker=dict(color='green', size=8, symbol='triangle-up'), name='Long entry'))
            if sdf['short_entry'].any():
                e = sdf[sdf['short_entry']]
                sfig.add_trace(go.Scatter(x=e['ts'], y=e['close'], mode='markers', marker=dict(color='red', size=8, symbol='triangle-down'), name='Short entry'))

            sfig.update_layout(title=f'Signals for {day} (dir={direction})', template='plotly_white', xaxis_rangeslider_visible=False, height=600)
            display(sfig)

            # Mini summary table
            import pandas as pd
            rows = []
            for idx, row in sdf[sdf['long_entry'] | sdf['short_entry']].iterrows():
                side = 'LONG' if row['long_entry'] else 'SHORT'
                stop = (lo if side=='LONG' else hi)
                tgt = (ext127 if side=='LONG' else ext127)
                rows.append({
                    'ts': row['ts'],
                    'side': side,
                    'price': row['close'],
                    'stop_level': stop,
                    'tgt_level': tgt,
                    'ema_fast': row['ema_fast'],
                    'ema_slow': row['ema_slow'],
                    'rsi': row['rsi'],
                })
            if rows:
                sigdf = pd.DataFrame(rows)
                display(sigdf.head(20))
            else:
                print('[signals] no entries under current rules')
except Exception as ex:
    print('[signals] error:', repr(ex))


[signals] dt=2025-03-11 rows=390


,ts,side,price,stop_level,tgt_level,ema_fast,ema_slow,rsi
0,2025-03-11 15:09:00+00:00,LONG,558.0300,552.02,567.284,556.540608,556.047605,67.819444
1,2025-03-11 15:10:00+00:00,LONG,557.4500,552.02,567.284,556.722487,556.175095,61.549561
2,2025-03-11 15:12:00+00:00,LONG,558.4300,552.02,567.284,557.215592,556.503632,67.366744
3,2025-03-11 15:13:00+00:00,LONG,558.1500,552.02,567.284,557.402473,556.653302,64.323625
4,2025-03-11 15:14:00+00:00,LONG,557.9000,552.02,567.284,557.501979,556.766638,61.646028
5,2025-03-11 15:15:00+00:00,LONG,558.4300,552.02,567.284,557.687583,556.917853,64.974743
6,2025-03-11 15:16:00+00:00,LONG,558.1000,552.02,567.284,557.770066,557.025321,61.401458
7,2025-03-11 15:17:00+00:00,LONG,558.6200,552.02,567.284,557.940053,557.170292,64.696180
8,2025-03-11 15:18:00+00:00,LONG,558.2700,552.02,567.284,558.006042,557.270265,60.926508
9,2025-03-11 15:19:00+00:00,LONG,558.3900,552.02,567.284,558.082834,557.372059,61.749434


In [ ]:
# === FIB GRID SEARCH (drop-in) ===============================================
import itertools, math, numpy as np, pandas as pd
from collections import defaultdict
from datetime import datetime

# --- helpers -------------------------------------------------
def simulate_day_with_params(day, ema_fast=9, ema_slow=21, rsi_len=14,
                             rsi_long=50, rsi_short=50, atr_len=14,
                             atr_mult_stop=0.5, retr_low=0.50, retr_high=0.618,
                             ext_pref=('161','127'), session_filter=None):
    df = load_day(day)
    if df.empty: 
        return pd.DataFrame()

    # indicators
    if ('ema_fast' not in df.columns) or ('ema_slow' not in df.columns):
        df['ema_fast'] = ema(df['close'], ema_fast)
        df['ema_slow'] = ema(df['close'], ema_slow)
    df['rsi'] = rsi(df['close'], rsi_len)
    df['atr'] = atr(df, atr_len)

    # Optional time-of-day filter: pass ("09:35","11:00","14:00","15:45") etc.
    if session_filter:
        def in_window(ts, windows):
            t = pd.to_datetime(ts).time()
            return any(start <= t <= end for (start, end) in windows)
        windows = []
        for s,e in zip(session_filter[::2], session_filter[1::2]):
            windows.append((pd.to_datetime(s).time(), pd.to_datetime(e).time()))
        df = df[df['ts'].apply(lambda x: in_window(x, windows))].reset_index(drop=True)
        if df.empty: 
            return pd.DataFrame()

    # NOTE: compute_fib_levels(df, direction='auto') likely uses full-day hi/lo → leakage.
    # Until you anchor to a rolling swing, we keep your approach but warn about it.
    fibc = compute_fib_levels(df, direction='auto')
    if not fibc: 
        return pd.DataFrame()

    lo, hi, direction = float(fibc['low']), float(fibc['high']), fibc['direction']

    # Compute retracement levels numerically to allow non-canonical ratios (e.g. 0.62)
    rl = max(0.0, min(1.0, float(retr_low)))
    rh = max(0.0, min(1.0, float(retr_high)))
    if direction == 'up':
        Llo = lo + (hi - lo) * rl
        Lhi = lo + (hi - lo) * rh
    else:
        Llo = hi - (hi - lo) * rl
        Lhi = hi - (hi - lo) * rh

    # extension targets from the same impulse
    rng = hi - lo
    ext127 = (hi + 0.272*rng) if direction=='up' else (lo - 0.272*rng)
    ext161 = (hi + 0.618*rng) if direction=='up' else (lo - 0.618*rng)
    ext_map = {'127': ext127, '161': ext161}

    # entries: retrace touches the zone with trend + RSI confirmation
    zone_lo, zone_hi = min(Llo, Lhi), max(Llo, Lhi)
    in_zone = (df['low'] <= zone_hi) & (df['high'] >= zone_lo)
    trend_up = df['ema_fast'] > df['ema_slow']
    trend_dn = df['ema_fast'] < df['ema_slow']
    df['long_entry']  = (direction=='up') & in_zone & trend_up  & (df['rsi'] >= rsi_long)
    df['short_entry'] = (direction=='down') & in_zone & trend_dn & (df['rsi'] <= (100 - rsi_short))

    trades, N = [], len(df)
    i, last_exit_idx = 0, -1
    while i < N:
        if i <= last_exit_idx: 
            i += 1; continue
        row = df.iloc[i]
        side = 'LONG' if row.get('long_entry', False) else ('SHORT' if row.get('short_entry', False) else None)
        if not side:
            i += 1; continue

        entry_idx = i + 1
        if entry_idx >= N: break
        entry_time = df.iloc[entry_idx]['ts']
        entry = float(df.iloc[entry_idx]['open'])
        atr_val = float(df.iloc[entry_idx]['atr'])
        raw_stop = float(lo) if side=='LONG' else float(hi)
        stop = raw_stop - atr_mult_stop*atr_val if side=='LONG' else raw_stop + atr_mult_stop*atr_val
        R = abs(entry - stop)
        if not np.isfinite(R) or R <= 0:
            i += 1; continue

        # management
        tp1 = entry + R if side=='LONG' else entry - R
        ext1, ext2 = ext_map[ext_pref[0]], ext_map[ext_pref[1]]
        stop_lvl = stop
        filled_half = False
        exit_idx, exit_reason, seq_exit_price = N-1, 'EOD', float(df.iloc[N-1]['close'])

        def hit_up(level,h,l):  return h >= level
        def hit_dn(level,h,l):  return l <= level

        for j in range(entry_idx+1, N):
            h = float(df.iloc[j]['high']); l = float(df.iloc[j]['low'])
            # stop first
            if (side=='LONG' and hit_dn(stop_lvl,h,l)) or (side=='SHORT' and hit_up(stop_lvl,h,l)):
                exit_idx, exit_reason, seq_exit_price = j, 'STOP', stop_lvl
                if filled_half:
                    pnl_first = 0.5*R
                    pnl_second = -0.5*R  # runner stopped at BE or below
                else:
                    pnl_first = pnl_second = -0.5*R
                break

            # first target → BE
            if not filled_half and ((side=='LONG' and hit_up(tp1,h,l)) or (side=='SHORT' and hit_dn(tp1,h,l))):
                filled_half = True
                pnl_first = 0.5*R
                stop_lvl = entry  # BE

            # extension targets
            if filled_half:
                if (side=='LONG' and hit_up(ext1,h,l)) or (side=='SHORT' and hit_dn(ext1,h,l)):
                    exit_idx, exit_reason, seq_exit_price = j, f'TP{ext_pref[0]}', ext1
                    gain = (ext1 - entry) if side=='LONG' else (entry - ext1)
                    pnl_second = 0.5*gain
                    break
                if (side=='LONG' and hit_up(ext2,h,l)) or (side=='SHORT' and hit_dn(ext2,h,l)):
                    exit_idx, exit_reason, seq_exit_price = j, f'TP{ext_pref[1]}', ext2
                    gain = (ext2 - entry) if side=='LONG' else (entry - ext2)
                    pnl_second = 0.5*gain
                    break
        else:
            # EOD
            if filled_half:
                pnl_first = 0.5*R
                pnl_second = 0.5*((df.iloc[N-1]['close'] - entry) if side=='LONG' else (entry - df.iloc[N-1]['close']))
            else:
                both = 0.5*((df.iloc[N-1]['close'] - entry) if side=='LONG' else (entry - df.iloc[N-1]['close']))
                pnl_first = pnl_second = both

        pnl_abs = pnl_first + pnl_second
        trades.append({
            'day': day, 'ts_in': entry_time, 'side': side, 'entry': entry,
            'R': R, 'exit_ts': df.iloc[exit_idx]['ts'], 'exit_reason': exit_reason,
            'exit_price': seq_exit_price, 'pnl_R': pnl_abs / R
        })
        last_exit_idx = exit_idx
        i = exit_idx + 1

    return pd.DataFrame(trades)

def summarize(df):
    if df.empty: 
        return {}
    wins = (df['pnl_R'] > 0).sum()
    losses = len(df) - wins
    netR = df['pnl_R'].sum()
    gross_win = df.loc[df['pnl_R']>0, 'pnl_R'].sum()
    gross_loss = -df.loc[df['pnl_R']<=0, 'pnl_R'].sum()
    pf = (gross_win / gross_loss) if gross_loss > 0 else np.inf
    # equity & drawdown
    eq = df['pnl_R'].cumsum()
    peak = eq.cummax()
    dd = (eq - peak)
    max_dd = dd.min()
    exp = df['pnl_R'].mean()
    return dict(n=len(df), wins=int(wins), losses=int(losses), netR=float(netR),
                pf=float(pf), maxDD=float(max_dd), expectancy=float(exp))

# --- grid ----------------------------------------------------
days = [
    # put your training dates here
    '2025-03-10','2025-03-11','2025-03-12'
]

param_grid = dict(
    ema_fast=[8,9,12],
    ema_slow=[21,26,34],
    rsi_len=[14],
    rsi_long=[48,50,52],     # long filter threshold
    rsi_short=[48,50,52],    # symmetric short filter
    atr_len=[14,21],
    atr_mult_stop=[0.25,0.5,0.75,1.0],
    retr_low=[0.50,0.382],
    retr_high=[0.618,0.62],  # keep near canonical values
    ext_pref=[('161','127'), ('127','161')]
)

rows = []
keys, vals = zip(*param_grid.items())
for combo in itertools.product(*vals):
    P = dict(zip(keys, combo))
    if P['ema_fast'] >= P['ema_slow']: 
        continue
    all_trades = []
    for d in days:
        tdf = simulate_day_with_params(d, **P, session_filter=("09:35","11:00","14:00","15:45"))
        if not tdf.empty:
            all_trades.append(tdf)
    if not all_trades:
        continue
    res = pd.concat(all_trades, ignore_index=True)
    metrics = summarize(res)
    rows.append({**P, **metrics})

results = pd.DataFrame(rows).sort_values(['netR','pf'], ascending=[False, False])
print(results.head(15).to_string(index=False))
# save full table if you like
# results.to_csv('fib_grid_results.csv', index=False)


KeyError: '61%'